In [1]:
from copy import deepcopy
import os

import fastplotlib as fpl
import numpy as np

import mesmerize_core as mc
import mesmerize_viz

using: <jupyter_rfb._jpg.SimpleJpegEncoder object at 0x7fc37c6e8b90>


2023-11-08 12:27:09.585983: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 12:27:09.617359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 12:27:09.617421: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 12:27:09.617475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 12:27:09.626930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 12:27:09.627632: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
imgs = [np.random.rand(100, 100, 100)] * 3
iw = fpl.ImageWidget(imgs)
iw.show()

RFBOutputContext()

/home/kushal/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [3]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

In [4]:
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

In [5]:
# Mac users!
# temporary patch for Mac, won't be necessary in next release
# Thanks Ryan Ly for the PR! :D I need to dig into it more before merging
# conda_prefix_1_str = os.environ['CONDA_PREFIX'].replace(os.path.join(' ', 'envs', 'mescore')[1:], '')
# os.environ['CONDA_PREFIX_1'] = conda_prefix_1_str

# Set `caiman_data` path

In [6]:
mc.set_parent_raw_data_path("/home/kushal/caiman_data/")

PosixPath('/home/kushal/caiman_data')

In [7]:
batch_path = mc.get_parent_raw_data_path().joinpath("minimal_test/batch.pickle")

In [8]:
# create a new batch, delete if exist (for test purpose only, not for your real batches!)
df = mc.create_batch(batch_path)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


In [9]:
mcorr_params = {
    "main":{
        'max_shifts': [12, 12],
        'strides': [32, 32],
        'overlaps': [20, 20],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None,
    }
}

In [10]:
df.caiman.add_item(
    algo="mcorr",
    input_movie_path=mc.get_parent_raw_data_path().joinpath("example_movies/demoMovie.tif"),
    params=mcorr_params,
    item_name="demo",
)

In [11]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,demo,example_movies/demoMovie.tif,"{'main': {'max_shifts': (12, 12), 'strides': (...",None,2023-11-08T12:27:27,None,None,None,dfc262c9-68e9-4dc4-bc30-dbc82bdba8f9


In [12]:
df.iloc[-1].caiman.run()

df = df.caiman.reload_from_disk()

2023-11-08 12:27:33.729508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 12:27:33.763265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 12:27:33.763307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 12:27:33.763337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 12:27:33.769890: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 12:27:33.770075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

starting mc


100%|██████████| 1/1 [00:00<00:00, 44.47it/s]


mc finished successfully!
computing projections
Computing correlation image
finished computing correlation image


In [13]:
viz = df.mcorr.viz(data_options=["input", "mcorr"], image_widget_kwargs={"grid_plot_kwargs": {"size": (700, 300)}})
viz.show()

/home/kushal/venvs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:460: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/kushal/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [14]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.9,
            'rf': 20,
            'stride': 12, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

In [15]:
df.caiman.add_item(
    algo="cnmf",
    input_movie_path=df.iloc[0],
    params=params_cnmf,
    item_name="demo",
)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,demo,example_movies/demoMovie.tif,"{'main': {'max_shifts': (12, 12), 'strides': (...",{'mean-projection-path': dfc262c9-68e9-4dc4-bc...,2023-11-08T12:27:27,2023-11-08T12:27:37,2.07 sec,None,dfc262c9-68e9-4dc4-bc30-dbc82bdba8f9
1,cnmf,demo,dfc262c9-68e9-4dc4-bc30-dbc82bdba8f9/dfc262c9-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2023-11-08T12:27:46,None,None,None,75f4f2f2-29e9-4e40-ab0a-9535d5fe12c1


In [16]:
df.iloc[-1].caiman.run()
df = df.caiman.reload_from_disk()

2023-11-08 12:27:51.838169: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 12:27:51.870187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 12:27:51.870224: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 12:27:51.870251: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 12:27:51.876665: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 12:27:51.876839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                                        demo
input_movie_path    dfc262c9-68e9-4dc4-bc30-dbc82bdba8f9/dfc262c9-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
added_time                                        2023-11-08T12:27:46
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             75f4f2f2-29e9-4e40-ab0a-9535d5fe12c1
Name: 1, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.9, 'rf': 20, 'stride': 12, 'K': 4, 'gSig': (4, 4), 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_c

2023-11-08 12:27:57.356169: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-11-08 12:27:57.356203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: kushal-p15v
2023-11-08 12:27:57.356213: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: kushal-p15v
2023-11-08 12:27:57.356514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-08 12:27:57.356564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.129.3
2023-11-08 12:27:57.356574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.129.3


performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/kushal/caiman_data/model/cnn_model.json
1/1 [==============================] - 0s 110ms/step


In [17]:
viz = df.cnmf.viz()
viz.show()

/home/kushal/venvs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:460: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

RFBOutputContext()

/home/kushal/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


RFBOutputContext()